<a href="https://colab.research.google.com/github/KhushMody/Image_classification/blob/master/CNN_image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! kaggle datasets download -d puneet6060/intel-image-classification
! unzip intel-image-classification

In [ ]:
import numpy as np
import pandas as pd
from keras import layers
from keras.layers import Input,Dense,Activation,ZeroPadding2D,BatchNormalization,Flatten,Conv2D
from keras.layers import AveragePooling2D,MaxPooling2D,Dropout,GlobalAveragePooling2D,GlobalMaxPooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
import scipy.misc
import imageio

import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

In [ ]:
def hmodel():
  num_classes = 6
  img_height,img_width = (150,150)
  X_input = Input(shape = (img_height,img_width,3))
  X = ZeroPadding2D((2,2))(X_input)
  X = Conv2D(16,(2,2),strides=(1,1), padding='same',name = 'conv0')(X)
  X = BatchNormalization(axis = 3,name = 'bn0')(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2,2),padding='same',name = 'max_pool0')(X)

  X = Conv2D(32,(2,2),strides=(1,1), padding = 'same', name = 'conv1')(X)
  X = BatchNormalization(axis = 3, name = 'bn1')(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2,2), padding = 'valid',name = 'max_pool1')(X)
  
  X = Conv2D(64,(2,2),strides=(1,1), padding = 'same', name = 'conv2')(X)
  X = BatchNormalization(axis = 3, name = 'bn2')(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2,2), padding = 'valid',name = 'max_pool2')(X)
  
  X = Conv2D(128,(2,2),strides=(1,1), padding = 'same', name = 'conv3')(X)
  X = BatchNormalization(axis = 3, name = 'bn3')(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2,2), padding = 'valid',name = 'max_pool3')(X)
  
  X = Flatten()(X)
  X = Dense(256)(X)
  X = BatchNormalization()(X)
  X = Activation('relu')(X)

  X = Dense(256)(X)
  X = BatchNormalization()(X)
  X = Activation('relu')(X)

  X_output = Dense(num_classes, activation = 'softmax')(X)

  model = Model(inputs = X_input, outputs = X_output)

  return model

In [ ]:
model = hmodel()

In [ ]:
model.compile(optimizer = 'adam', loss= 'categorical_crossentropy', metrics = ['accuracy'])
model.summary()

In [ ]:
img_size = 150
BS = 32

datagen_train = ImageDataGenerator(horizontal_flip = True,
                                   rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2)
train_genrator = datagen_train.flow_from_directory("/content/seg_train/seg_train",
                                                   target_size = (img_size,img_size),
                                                   batch_size = BS,
                                                   class_mode = "categorical",
                                                   shuffle = True)

datagen_validation = ImageDataGenerator(horizontal_flip = True,
                                        rescale = 1./255,
                                        shear_range = 0.2,
                                        zoom_range = 0.2)
validation_generator = datagen_validation.flow_from_directory("/content/seg_test/seg_test",
                                                              target_size = (img_size,img_size),
                                                              batch_size = BS,
                                                              class_mode = "categorical",
                                                              shuffle = False)


In [ ]:
epochs = 10
steps_per_epoch = train_genrator.n//train_genrator.batch_size
validation_steps = validation_generator.n//validation_generator.batch_size

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor=0.1, patience= 2, min_lr = 0.00001, mode = 'auto')
checkpoint = ModelCheckpoint("model_weights.h5", monitor='val_accuracy',save_weights_only=True, mode = 'max',verbose = 1)
callbacks = [checkpoint, reduce_lr]

H = model.fit(x = train_genrator,
              steps_per_epoch = steps_per_epoch,
              epochs = epochs,
              validation_data = validation_generator,
              validation_steps = validation_steps,
              #callbacks = callbacks
              )

In [ ]:
model.save("./image_classification.h5")

H.history

In [ ]:
img_path = '/content/black-high-rise-building-under-grey-and-white-sky-during-65438.jpg'
img = image.load_img(img_path, target_size=(150, 150))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = x/255.0
print('Input image shape:', x.shape)
my_image = imageio.imread(img_path)
imshow(my_image)
#print("class prediction vector [p(0), p(1), p(2), p(3), p(4), p(5)] = ")
print(np.max(model.predict(x)))